# Data structures

**Author(s)**: Jukka Aho

**Abstract**: Description of data structures.

## Revision history

### 2015-06-14
- Initial version.

### 2015-09-25
- Complete rewrite. The main ideas proposed in earlier version didn't work.

## Data fields on elements

Typical element structure so far:

    type MySuperElement <: Element
        connectivity :: Array{Int, 1}  # describes how dofs of this element is connected to another elements
        fields :: ???
    end

- Fields must be interpolable, in space $\mathbb{C}^n \times \mathbb{R}$, i.e. $f(\boldsymbol{\xi}, t) = \sum_i \phi_i(\boldsymbol{\xi}) f_i(t) = \sum_i \phi_i(\boldsymbol{\xi}) \sum_j \varphi_j(t) f_{ij}$, where $f_{ij}$ is scalar, tensor or vector defined in element area $e$ by some basis functions $\phi(\boldsymbol{\xi})$ and $\varphi(t)$. Parameter $t$ is normally considered as "time" and $\xi$ is dimensionless coordinate. Parameter $t$ has not necessarily to be time, it could be for example angle $\alpha \in [-2\pi, 2\pi]$ or similar.
- We store mainly three fields, scalar field, vector field, tensor field. Field may or may not be dependent from parameters $\xi$ or $t$.
- $t$ is discretized to several steps $\{t_0, t_1, \ldots, t_n\}$. Each discrete time $t_i$ may contain several iterations until convergence. We want to save and get access to all of this data if needed.
- So in practice we have a set of fields $f(\boldsymbol{\xi})$ over time domain $t$. Typically some fields, like Geometry, is introduced only in time $t_0$. Some other fields, like boundary load, may be "active" only on some time $\hat{t} \subset t$. Some care must be taken about how to extrapolate field variables before definition and after them. Considering for example boundary conditions is feels quite natural to say that before boundary load $g$ is introduced the field variable equals to zero. And if field is not defined in some time $t$ we could use the last known value, for example if we ramp up some load $g(t) = t, t\in[0,1]$ we could say that $g(t) = g(1)$ if $t > 1$.
- In the simplest case (simple nonlinear quasistatic analysis), we have for instance $t \in [0, 1]$ where boundary conditions are set in $t_0$ and load is set in $t_1$. We may use adaptive strategies to shorten time if convergence issues araises.

## Without time domain

Without time we have something like this

In [1]:
using ForwardDiff

In [2]:
type Field{T}
    name
    values :: Array{T, 1}
end

type Basis
    basis :: Function
    dbasis :: Function
end

In [3]:
function Basis(basis)
    Basis(basis, ForwardDiff.jacobian(basis))
end
call(b::Basis, xi) = b.basis(xi)
Base.(:*)(x::Array{Float64, 1}, f::Field) = sum(x .* f.values)
Base.(:*)(b::Basis, f::Field) = (x) -> b(x)*f

* (generic function with 158 methods)

Then we can do something like this

In [4]:
N = Basis((ξ) -> [0.5*(1.0-ξ[1]), 0.5*(1.0+ξ[1])])
u = Field(:displacement, [0.0, 1.0])

Field{Float64}(:displacement,[0.0,1.0])

In [5]:
N([0.0])*u

0.5

or

In [6]:
(N*u)([0.0])

0.5

## Extending to time domain

Next we want to interpolate over time domain. Maybe something like this would do the job:

In [7]:
Base.(:*)(k::Float64, f::Field) = Field(f.name, k*f.values)
u1 = Field(:displacement, [0.0, 1.0])
3.0*u1

Field{Float64}(:displacement,[0.0,3.0])

In [8]:
function Base.(:+)(f1::Field, f2::Field)
    @assert(f1.name == f2.name, "Cannot add field $(f1.name) to $(f2.name): field name mismatch")
    Field(f1.name, f1.values + f2.values)
end
u1 = Field(:displacement, [0.0, 1.0])
u2 = Field(:displacement, [1.0, 2.0])
u1 + u2

Field{Float64}(:displacement,[1.0,3.0])

In [9]:
u1 = Field(:displacement, [0.0, 1.0])
u2 = Field(:displacement, [1.0, 2.0])

t = Basis((t) -> [1-t, t])
u = Field[u1, u2]
Base.(:*)(x::Array{Float64, 1}, f::Array{Field}) = sum(x .* f)
Base.(:*)(b::Basis, f::Array{Field}) = (t) -> b(t)*f
(t*u)(0.5)

Field{Float64}(:displacement,[0.5,1.5])

## Summary

Putting this together so far:

In [20]:
ϕ = Basis((ξ) -> [0.5*(1.0-ξ[1]), 0.5*(1.0+ξ[1])])
φ = Basis((t) -> [1-t, t])
u1 = Field(:displacement, [0.0, 1.0])
u2 = Field(:displacement, [1.0, 2.0])
# interpolate displacement u1 in mid-point ξ=[0.0] on element
(ϕ*u1)([0.0]) # => 0.5
# interpolate displacement field [u1, u2] in time t=0.5
(φ*Field[u1, u2])(0.5) # => Field{Float64}(:displacement,[0.5,1.5])
# interpolate displacement in element area ξ and time t
d(ξ, t) = (ϕ*(φ*Field[u1, u2])(t))(ξ)
d([0.0], 0.5)

1.0

Makes sense, since deformation from $u_1$ to $u_2$ at $t=0.5$ is $\begin{bmatrix}0.5 & 1.5\end{bmatrix}$ and taking the midpoint of this makes exactly $1.0$.

In [ ]:
type Iteration{T}
    converged :: Bool
    values :: T
end

type Increment
    time :: Float64
    iterations :: Array{Iteration, 1}
end

type Step
    f :: Function
    t :: Array{Float64, 1}
    increments :: Array{Increment, 1}
end

In [ ]:
function Iteration()
    Iteration(false, 
end
f = Field(:temperature, 

In [37]:
f(0.0)

LoadError: LoadError: MethodError: `call` has no method matching call(::Field, ::Float64)
Closest candidates are:
  Union(!Matched::Any...)
  BoundsError()
  BoundsError(!Matched::Any...)
  ...
while loading In[37], in expression starting on line 1

In [39]:
interpolate(f, ϕ, [0.0])

0.5

In [40]:
(ϕ*f)([0.0])

0.5